In [407]:
import database
import req
import pprint
import json
import os
import csv
pwd = os.getcwd()
pp = pprint.PrettyPrinter(indent=4)

dbsavefile = '/DBsave.json'
#DB = database.DB_init()

def saveDB(db):
    with open("".join([pwd,dbsavefile]), 'wb') as jsonfile: 
            json.dump(db,jsonfile)

def num(s):
    try:
        return int(s)
    except ValueError:
        try:
            return float(s)
        except ValueError:
            return s

def d_or_v(val):
    if isinstance(val,dict):
        return key_to_int(val)
    else:
        return val
   
def key_to_int(d):
    return {num(k):d_or_v(v) for k,v in d.iteritems()}   

def loadDB():
    with open("".join([pwd,dbsavefile]), 'rb') as jsonfile: 
        db = json.load(jsonfile)
    return key_to_int(db)

    

DB=loadDB()
pprint = pp.pprint
Teams = DB['Teams']
Games = DB['Games']



In [232]:

def print_team(d,score = 0.0):
    pprint(''.join([str(d['id']),': ',d['name'],'  strength = ',str(score)]))

def print_game(d):
    if d['status']==1:
        pprint("".join([str(d['id']),':', 'r', str(d['round']),'  ',
                        str(d['scores'][d['team_1']]),' -- ',
                        DB['Teams'][d['team_1']]['name'],
                        ' vs ', DB['Teams'][d['team_2']]['name'], ' -- ',
                        str(d['scores'][d['team_2']])]))
    else:
        pprint("".join([str(d['id']),':','r',str(d['round']),'  ',
                DB['Teams'][d['team_1']]['name'],
                ' vs ', DB['Teams'][d['team_2']]['name']]))
def list_teams(session_id):
    session = DB['Sessions'][session_id]
    for id in session['teams']:
        s = session['strengths'][id]
        print_team(DB['Teams'][id],score = s)
def list_games(session_id):
    session = DB['Sessions'][session_id]
    for id in session['games']:
        print_game(DB['Games'][id])

        
def games_played(team_id,session_id):
    def count(g,t):
        if DB['Games'][g]['status'] == 1 and t in [DB['Games'][g]['team_1'],DB['Games'][g]['team_2']]:
            return 1
        else:
            return 0
        
    return sum([count(g_id,team_id) for g_id in DB['Sessions'][session_id]['games']])
    
def export_results(session_id):
    session = DB['Sessions'][session_id]
    s = session['strengths']
    
    d = sorted([[id,DB['Teams'][id]['name'],
          "{0:.2f}".format(session['strengths'][id]),
         games_played(id,session_id)] for id in session['teams']],key = lambda x: s[x[0]])

    with open("".join([pwd,'/results.csv']), 'wb') as csvfile:

        writer = csv.writer(csvfile,delimiter='\t')
    
        for line in d:
            writer.writerow(line)

            
def export_games(session_id):
    session = DB['Sessions'][session_id]
    played_games = [Games[id] for id in session['games'] if Games[id]['status'] == 1]
    unplayed_games = [Games[id] for id in session['games'] if not Games[id]['status'] == 1]

    pd = [[g['id'],Teams[g['team_1']]['name'],Teams[g['team_2']]['name'],
            g['round'],g['scores'][g['team_1']],
            g['scores'][g['team_2']]] for g in played_games]    
    upd = [[g['id'],Teams[g['team_1']]['name'],Teams[g['team_2']]['name'],
            g['round'],'none',
            'none'] for g in unplayed_games]    
    with open("".join([pwd,'/games.csv']), 'wb') as csvfile:

        writer = csv.writer(csvfile,delimiter='\t')
    
        for line in pd:
           writer.writerow(line)
        
        for line in upd:
           writer.writerow(line)
        
def export_suggested_games(session_id):
    session = DB['Sessions'][session_id]
    suggested_games = session['suggestedGames'].values()

    pd = [[g['id'],Teams[g['team_1']]['name'],Teams[g['team_2']]['name'],
            g['round']] for g in suggested_games]    

    with open("".join([pwd,'/suggested_games.csv']), 'wb') as csvfile:

        writer = csv.writer(csvfile,delimiter='\t')
    
        for line in pd:
           writer.writerow(line)
        
def export_teams_list(session_id):
    session = DB['Sessions'][session_id]
    teams = [DB['Teams'][id] for id in session['teams']]

    pd = [[t['id'],t['name']] for t in teams]    

    with open("".join([pwd,'/team_list.csv']),'wb') as csvfile:

        writer = csv.writer(csvfile,delimiter='\t')
    
        for line in pd:
           writer.writerow(line)
        




In [452]:
sess_id = 5
export_teams_list(sess_id)
export_games(sess_id)
export_results(sess_id)

def compute_strengths():
    req.update_strengths(DB,sess_id)
    export_results(sess_id)

def stage_round(round = 0):
    req.get_suggested_games(DB,sess_id,round)
    export_suggested_games(sess_id)
    print('done')
    
def update_score(game_id=-1,score_1 = -1, score_2 = -1):
    req.update_score(DB,game_id,score_1,score_2)
    export_games(sess_id)
    print_game(Games[game_id])

def add_suggested_games():
    req.add_suggested_games(DB,sess_id)
    export_games(sess_id)
def add_suggested_game(game_id=0):
    req.add_game(DB,sess_id,DB['Sessions'][sess_id]['suggestedGames'][game_id])
    export_games(sess_id)

def add_game(team_1 = 0, team_2 = 0, round = 0):
    req.add_game(DB,sess_id,req.new_game(team_1,team_2,round))
    export_games(sess_id)

def remove_game(game_id=-1):
    req.remove_game(DB, sess_id, game_id)
    export_games(sess_id)

    
def check_teams(team_1=0,team_2=0):
    games = [Games[id] for id in DB['Sessions'][sess_id]['games']]
    for game in games:
        if set([game['team_1'],game['team_2']]) == set([team_1,team_2]):
            print('played')
            return True
    print('unplayed')
    return False

In [461]:
#Add a game to the session. Use the team ID to specify the teams
add_game(team_1 = 28, team_2 = 16, round = 1 )

In [353]:
check_teams(team_1=12,team_2=25)

unplayed


False

In [390]:
# update the scores in a game. get the game id from the list of games
update_score(game_id=156,score_1 =7, score_2 =0)

u'156:r9  7 -- Three Boys vs Kumaras bravas -- 0'


In [359]:
# remove a game from the session
remove_game(game_id=155)

In [391]:
# Compute the strengths of the teams. This may take some time.
compute_strengths()

[2, 1, 8, 11, 0, 4, 6, 15, 19, 16, 21, 18, 22, 27, 2, 1, 11, 0, 4, 8, 6, 1, 0, 4, 2, 11, 6, 8, 2, 11, 0, 4, 8, 6, 1, 0, 1, 11, 2, 4, 8, 6, 16, 19, 21, 15, 27, 18, 22, 21, 15, 16, 19, 18, 22, 27, 18, 16, 19, 15, 21, 27, 22, 27, 15, 16, 18, 19, 21, 22, 2, 19, 1, 16, 11, 18, 6, 15, 8, 21, 0, 22, 4, 27, 16, 6, 2, 1, 11, 19, 8, 15, 18, 21, 22, 4, 0, 27, 2, 16, 1, 19, 11, 6, 22, 8, 27, 15, 18, 0, 4, 21, 0, 21, 1, 27, 2, 22, 4, 19, 6, 18, 8, 16, 11, 15]
[3, 5, 7, 9, 10, 12, 13, 14, 17, 20, 23, 24, 25, 26]
running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 54.5 sec

In [364]:
#Suggest games for the next round. This may take quite some time.
stage_round(round  =9)

[2, 1, 8, 11, 0, 4, 6, 15, 19, 16, 21, 18, 22, 27, 2, 1, 11, 0, 4, 8, 6, 1, 0, 4, 2, 11, 6, 8, 2, 11, 0, 4, 8, 6, 1, 0, 1, 11, 2, 4, 8, 6, 16, 19, 21, 15, 27, 18, 22, 21, 15, 16, 19, 18, 22, 27, 18, 16, 19, 15, 21, 27, 22, 27, 15, 16, 18, 19, 21, 22, 2, 19, 1, 16, 11, 18, 6, 15, 8, 21, 0, 22, 4, 27, 16, 6, 2, 1, 11, 19, 8, 15, 18, 21, 22, 4, 0, 27, 2, 16, 1, 19, 11, 6, 22, 8, 27, 15, 18, 0, 4, 21, 0, 21, 1, 27, 2, 22, 4, 19, 6, 18, 8, 16, 11, 15]
[3, 5, 7, 9, 10, 12, 13, 14, 17, 20, 23, 24, 25, 26]
running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 54.6 sec

Process Process-33:
Process Process-34:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
    self._target(*self._args, **self._kwargs)
  File "ranking.py", line 22, in parfun
  File "ranking.py", line 22, in parfun
    q_out.put((i,f(x)))
    q_out.put((i,f(x)))
  File "ranking.py", line 75, in kl
  File "ranking.py", line 71, in kl
    for s1 in scores ] for s2 in scores])
    for s1 in scores] for s2 in scores])
  File "ranking.py", line 68, in expect
  File "ranking.py", line 68, in expect
    return np.mean(fn(trace[:,ind]))
    return np.mean(fn(trace[:,ind]))
  File "ranking.py", line

KeyboardInterrupt: 

In [365]:
#This will add the suggested games to the list of games
add_suggested_games()

In [346]:
#add one of the suggested games to the list of games
add_suggested_game(game_id=2)

unplayed


False

In [307]:
saveDB(DB)

In [233]:
export_results(sess_id)

In [224]:
export_games(sess_id)

In [198]:
export_games(1)

In [372]:
import bayes
reload(req)
reload(bayes)
import ranking
reload(ranking)

<module 'ranking' from 'ranking.pyc'>

unplayed


NameError: global name 'false' is not defined

In [42]:
DB['Sessions'][3]['games'] = DB['Sessions'][1]['games'] + DB ['Sessions'][4]['games']
DB['Sessions'][4]['games'] = DB['Sessions'][1]['games'] + DB ['Sessions'][4]['games']

DB['Sessions'][3]['teams'] = [2,19,1,16,11,18,6,15,8,21,0,22,4,27]
DB['Sessions'][4]['teams'] = [24,13,14,9,25,7,17,12,23,3,20,10,26,5]

In [270]:
DB['Sessions'][4]['games'] = list(set(DB['Sessions'][3]['games']+DB['Sessions'][4]['games']))

In [374]:
import bayes
import ranking

teams = bayes.make_teams(DB['Sessions'][4]['teams'])
sess = bayes.init_session(teams)
sess.games = bayes.mcmc_games([DB['Games'][id] for id in DB['Sessions'][4]['games']],teams)


[2, 1, 8, 11, 0, 4, 6, 15, 19, 16, 21, 18, 22, 27, 2, 1, 11, 0, 4, 8, 6, 1, 0, 4, 2, 11, 6, 8, 2, 11, 0, 4, 8, 6, 1, 0, 1, 11, 2, 4, 8, 6, 16, 19, 21, 15, 27, 18, 22, 21, 15, 16, 19, 18, 22, 27, 18, 16, 19, 15, 21, 27, 22, 27, 15, 16, 18, 19, 21, 22, 2, 19, 1, 16, 11, 18, 6, 15, 8, 21, 0, 22, 4, 27, 16, 6, 2, 1, 11, 19, 8, 15, 18, 21, 22, 4, 0, 27, 2, 16, 1, 19, 11, 6, 22, 8, 27, 15, 18, 0, 4, 21, 0, 21, 1, 27, 2, 22, 4, 19, 6, 18, 8, 16, 11, 15]
[3, 5, 7, 9, 10, 12, 13, 14, 17, 20, 23, 24, 25, 26]


In [375]:
sess.fit_model()


running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 54.7 sec

In [143]:
"{0:.2f}".format(a)


[0, 1, 3, 5, 7, 10, 14, 15, 17, 18, 20, 21, 26, 27, 29, 31, 33, 36, 40, 41, 43, 44, 46, 47, 52, 54, 56, 58, 61, 65, 66, 68, 69, 71, 72, 77, 103, 105, 108, 112, 113, 115, 116, 118, 119, 124, 148, 151, 155, 156, 158, 159, 161, 162, 167, 190, 194, 195, 197, 198, 200, 201, 206, 245, 246, 248, 249, 251, 252, 257, 300, 302, 303, 305, 306, 311, 313, 314, 316, 317, 322, 333, 335, 336, 341, 343, 344, 349, 357, 362, 367]


In [376]:
kl = sess.kl_info_vec()

[79, 81, 83, 84, 86, 87, 88, 91, 94, 97, 98, 99, 100, 126, 128, 129, 131, 132, 133, 136, 139, 142, 143, 144, 145, 169, 170, 172, 173, 174, 177, 180, 183, 184, 185, 186, 207, 209, 210, 211, 214, 217, 220, 221, 222, 223, 226, 227, 228, 231, 234, 237, 238, 239, 240, 258, 259, 262, 265, 268, 269, 270, 271, 273, 276, 279, 282, 283, 284, 285, 289, 292, 295, 296, 297, 298, 325, 328, 329, 330, 331, 352, 353, 354, 355, 368, 369, 370, 372, 373, 375]


In [240]:
sg = sess.stage_round(9,kl)

[[2, 9], [1, 13], [7, 10], [12, 8], [11, 0], [3, 4], [6, 5], [15, 25], [19, 24], [16, 21], [17, 18], [23, 14], [20, 22], [26, 27], [13, 2], [1, 9], [7, 12], [11, 3], [0, 4], [8, 5], [6, 10], [1, 12], [13, 0], [9, 4], [2, 11], [5, 7], [6, 8], [3, 10], [2, 7], [11, 13], [0, 10], [4, 5], [3, 8], [12, 9], [6, 1], [0, 12], [1, 11], [2, 10], [4, 8], [5, 9], [13, 7], [6, 3], [24, 16], [19, 21], [14, 25], [15, 23], [20, 26], [17, 27], [18, 22], [21, 15], [24, 25], [16, 19], [20, 14], [18, 23], [17, 26], [22, 27], [20, 25], [18, 24], [23, 16], [14, 19], [17, 15], [21, 27], [26, 22], [14, 27], [15, 26], [16, 25], [17, 24], [18, 20], [19, 23], [21, 22], [2, 19], [1, 16], [11, 18], [6, 15], [8, 21], [0, 22], [4, 27], [16, 6], [2, 1], [11, 19], [8, 15], [18, 21], [22, 4], [0, 27], [2, 16], [1, 19], [11, 6], [22, 8], [27, 15], [18, 0], [4, 21]]


In [272]:
sess.games[1].teams[0].id

5

In [116]:
import networkx as nx


[2, 9]

In [377]:
kl

[{'t1': 3, 't2': 5, 'weight': 0.65755606172977288},
 {'t1': 3, 't2': 7, 'weight': 0.28291531421372973},
 {'t1': 3, 't2': 9, 'weight': 0.65739725038030317},
 {'t1': 3, 't2': 10, 'weight': 0.61963651383695506},
 {'t1': 3, 't2': 12, 'weight': 0.45181971355212447},
 {'t1': 3, 't2': 13, 'weight': 0.06017572176421071},
 {'t1': 3, 't2': 14, 'weight': 1.2706118120200185},
 {'t1': 3, 't2': 17, 'weight': 0.46754267301143182},
 {'t1': 3, 't2': 20, 'weight': 0.47935738115702708},
 {'t1': 3, 't2': 23, 'weight': 0.50020920703107397},
 {'t1': 3, 't2': 24, 'weight': 0.12925136413117944},
 {'t1': 3, 't2': 25, 'weight': 0.23700356508793563},
 {'t1': 3, 't2': 26, 'weight': 0.39631276105718971},
 {'t1': 5, 't2': 7, 'weight': 0.65119804051288543},
 {'t1': 5, 't2': 9, 'weight': 1.0214900964932667},
 {'t1': 5, 't2': 10, 'weight': 1.0543956291238805},
 {'t1': 5, 't2': 12, 'weight': 0.88326052158818946},
 {'t1': 5, 't2': 13, 'weight': 0.20935278339641306},
 {'t1': 5, 't2': 14, 'weight': 1.7758902622097614},
 {

In [329]:
x['Sessions'][2]['strengths']

{0: 1.240902108482105,
 1: -1.7313163712697726,
 2: -2.8765658394250044,
 3: 1.1288194903466013,
 4: 1.4651319018689322,
 5: 2.4161783699511941,
 6: 0.14419265791853592,
 7: -0.053151918012899063,
 8: 0.96596172161713834,
 9: -1.1502022887057295,
 10: 1.2421464553926431,
 11: -1.0857910719168931,
 12: 0.40310069577556018,
 13: -2.1094059120224267,
 14: 0,
 15: 0,
 16: 0,
 17: 0,
 18: 0,
 19: 0,
 20: 0,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0}

In [115]:
DB['Sessions'][4]['games'] = list(set(DB['Sessions'][4]['games']
                                 + DB['Sessions'][1]['games']
                                 + DB ['Sessions'][2]['games']))

In [2]:
pwd

'/home/ubuntu/bayesian/python'

In [16]:

DB['Sessions'][3]['teams'] = [2,19,1,16,11,18,6,15,8,21,0,22,4,27]
DB['Sessions'][4]['teams'] = [24,13,14,9,25,7,17,12,23,3,20,10,26,5]
DB['Sessions'][3]['strengths'] = {k:0.0 for k in DB['Sessions'][3]['teams']}
DB['Sessions'][4]['strengths'] = {k:0.0 for k in DB['Sessions'][4]['teams']}

In [144]:
saveDB(DB)

In [51]:
list_teams(3)

u'2: Golem  strength = 0.0'
u'19: Beware  strength = 0.0'
u'1: Wabbit  strength = 0.0'
u'16: The Jamesons  strength = 0.0'
u'11: Triple Fisted  strength = 0.0'
u'18: Mammoth  strength = 0.0'
u'6: Gentlemans  strength = 0.0'
u'15: The Royales with Cheese  strength = 0.0'
u'8: Rusty Spoons  strength = 0.0'
u'21: Lost Cat  strength = 0.0'
u'0: Ball Bangers  strength = 0.0'
u'22: Mighty Bisons  strength = 0.0'
u'4: JKL  strength = 0.0'
u"27: The Caracal's  strength = 0.0"


In [19]:
list_teams(4)

u'24: Three Boys  strength = 0.0'
u'13: Pete un Plomb  strength = 0.0'
u'14: Kitten Pile Crush  strength = 0.0'
u'9: Daft Monks  strength = 0.0'
u'25: Gobshites  strength = 0.0'
u'7: Sons of Odin  strength = 0.0'
u'17: Elf Death  strength = 0.0'
u'12: With Fire  strength = 0.0'
u'23: Sometimes  strength = 0.0'
u'3: The Edinburgh Badgers  strength = 0.0'
u'20: Two and a half Men  strength = 0.0'
u'10: The Budgie Smugglers  strength = 0.0'
u'26: Kumaras bravas  strength = 0.0'
u'5: The Inseminators  strength = 0.0'


In [55]:
export_games(4)

In [395]:
import ranking

teamids = bayes.make_teams(DB['Sessions'][1]['teams'] + DB['Sessions'][2]['teams'])

sess = bayes.init_session(teamids)
sess.games = bayes.mcmc_games([DB['Games'][id] for id in DB['Sessions'][4]['games']],teams)
sess.fit_model()
st = sess.strengths()
fac = sess.mcmc.scale.stats()['mean']
g_fun = lambda s1,s2,delta:ranking.log_2_pois_like(delta,st,s1,s2)
[g_fun(g['scores'][g['team_1']],g['scores'][g['team_2']],st[g['team_1']]-st[g['team_2']]) for g in DB['Games'].values()]


[2, 1, 8, 11, 0, 4, 6, 15, 19, 16, 21, 18, 22, 27, 2, 1, 11, 0, 4, 8, 6, 1, 0, 4, 2, 11, 6, 8, 2, 11, 0, 4, 8, 6, 1, 0, 1, 11, 2, 4, 8, 6, 16, 19, 21, 15, 27, 18, 22, 21, 15, 16, 19, 18, 22, 27, 18, 16, 19, 15, 21, 27, 22, 27, 15, 16, 18, 19, 21, 22, 2, 19, 1, 16, 11, 18, 6, 15, 8, 21, 0, 22, 4, 27, 16, 6, 2, 1, 11, 19, 8, 15, 18, 21, 22, 4, 0, 27, 2, 16, 1, 19, 11, 6, 22, 8, 27, 15, 18, 0, 4, 21, 0, 21, 1, 27, 2, 22, 4, 19, 6, 18, 8, 16, 11, 15]
[3, 5, 7, 9, 10, 12, 13, 14, 17, 20, 23, 24, 25, 26]
running MCMC
 [-----------------100%-----------------] 100000 of 100000 complete in 54.5 sec

AttributeError: log

In [403]:
[(g['scores'][g['team_1']],g['scores'][g['team_2']],
  st[g['team_1']]-st[g['team_2']]) for g in DB['Games'].values()]

KeyError: 'scores'

In [ ]:
teams = bayes.make_teams(DB['Sessions'][3]['teams'])
sess = bayes.init_session(teams)
sess.games = bayes.mcmc_games([DB['Games'][id] for id in DB['Sessions'][3]['games']],teams)

In [392]:
DB['Games'][0]

{u'id': 0,
 u'round': 1,
 u'scores': {2: 6, 9: 2},
 u'status': 1,
 u'team_1': 2,
 u'team_2': 9}

In [406]:
[g for g in DB['Games'].values()]

[{u'id': 0,
  u'round': 1,
  u'scores': {2: 6, 9: 2},
  u'status': 1,
  u'team_1': 2,
  u'team_2': 9},
 {u'id': 1,
  u'round': 1,
  u'scores': {1: 2, 13: 4},
  u'status': 1,
  u'team_1': 1,
  u'team_2': 13},
 {u'id': 2,
  u'round': 1,
  u'scores': {7: 3, 10: 2},
  u'status': 1,
  u'team_1': 7,
  u'team_2': 10},
 {u'id': 3,
  u'round': 1,
  u'scores': {8: 3, 12: 5},
  u'status': 1,
  u'team_1': 12,
  u'team_2': 8},
 {u'id': 4,
  u'round': 1,
  u'scores': {0: 1, 11: 11},
  u'status': 1,
  u'team_1': 11,
  u'team_2': 0},
 {u'id': 5,
  u'round': 1,
  u'scores': {3: 2, 4: 1},
  u'status': 1,
  u'team_1': 3,
  u'team_2': 4},
 {u'id': 6,
  u'round': 1,
  u'scores': {5: 2, 6: 5},
  u'status': 1,
  u'team_1': 6,
  u'team_2': 5},
 {u'id': 7,
  u'round': 1,
  u'scores': {15: 1, 25: 5},
  u'status': 1,
  u'team_1': 15,
  u'team_2': 25},
 {u'id': 8,
  u'round': 1,
  u'scores': {19: 2, 24: 3},
  u'status': 1,
  u'team_1': 19,
  u'team_2': 24},
 {u'id': 9,
  u'round': 1,
  u'scores': {16: 4, 21: 2},


In [424]:
[[t['id'],t['name']] for t in DB['Teams'].values()]

[[0, u'Ball Bangers'],
 [1, u'Wabbit'],
 [2, u'Golem'],
 [3, u'The Edinburgh Badgers'],
 [4, u'JKL'],
 [5, u'The Inseminators'],
 [6, u'Gentlemans'],
 [7, u'Sons of Odin'],
 [8, u'Rusty Spoons'],
 [9, u'Daft Monks'],
 [10, u'The Budgie Smugglers'],
 [11, u'Triple Fisted'],
 [12, u'With Fire'],
 [13, u'Pete un Plomb'],
 [14, u'Kitten Pile Crush'],
 [15, u'The Royales with Cheese'],
 [16, u'The Jamesons'],
 [17, u'Elf Death'],
 [18, u'Mammoth'],
 [19, u'Beware'],
 [20, u'Two and a half Men'],
 [21, u'Lost Cat'],
 [22, u'Mighty Bisons'],
 [23, u'Sometimes'],
 [24, u'Three Boys'],
 [25, u'Gobshites'],
 [26, u'Kumaras bravas'],
 [27, u"The Caracal's"],
 [28, u'3UP'],
 [30, u'Royals'],
 [31, u'Les Beehive Boys'],
 [32, u'The Control'],
 [33, u'Frenemies'],
 [34, u'Prospectors'],
 [35, u'TBD'],
 [36, u'TEMOILESNICHONS'],
 [37, u'LOS BIGOTES HASTA LAS RODILLAS'],
 [38, u'YATTARUS'],
 [39, u'The Huntsmen'],
 [40, u'Majestic'],
 [41, u'B'],
 [42, u'Call Me Daddy'],
 [43, u'Thunder fart'],
 [44, 

In [445]:
DB['Sessions'][5]['teams'] = [28,41,19,33,2,14,31,37,40,13,34,30,35,36,32,39,16,38]
DB['Sessions'][6]['teams'] = [55,42,46,9,54,25,47,49,50,51,45,44,52,24,43,11,1,53]
DB['Sessions'][5]['strengths']={}
DB['Sessions'][6]['strengths']={}

for id in DB['Sessions'][5]['teams']:
    DB['Sessions'][5]['strengths'][id]=0.0
    
for id in DB['Sessions'][6]['teams']:
    DB['Sessions'][5]['strengths']['id']=0.0

In [450]:
DB['Teams'][41]['name'] = 'Auckland Justice'

In [428]:
DB['Teams'][56] = {'id': 46,
 'name': u'Fully Torqued',
 'session': 'B',
 'strength': 0,
 'wc': False}

In [444]:
DB['Sessions'][5]

{u'games': [],
 u'id': 5,
 u'name': u'Worlds: Day 1 AM',
 u'strengths': {53: 0.0},
 u'suggestedGames': [],
 u'teams': [28,
  41,
  19,
  33,
  2,
  14,
  31,
  37,
  40,
  13,
  34,
  30,
  35,
  36,
  32,
  39,
  16,
  38],
 u'type': u'swiss'}

In [462]:
saveDB(DB)